# Clasificación con data de hundimiento del Titanic

In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('pyspark-titanic-sol').getOrCreate()

In [ ]:
spark.version

In [ ]:
dft = spark.read.csv('/dataset/titanic.csv', header=True)

In [ ]:
dft.show(10)

In [ ]:
dft.printSchema()

In [ ]:
from pyspark.sql.types import DoubleType, IntegerType

In [ ]:
dft = dft.withColumn('survived', dft['survived'].cast(IntegerType()))

In [ ]:
dft.printSchema()

In [ ]:
dft.select('survived').show(5)

In [ ]:
dft.groupBy('survived').count().orderBy('count').show(10)

## Extraccion (Ejercicio 1)

In [ ]:
from pathlib import Path
import pandas as pd

In [ ]:
def extract_titanic_data(url, refresh_cache=False):
    cache_fn = Path('/dataset/titanic_local.csv')
    if not cache_fn.exists() or refresh_cache:
        print("Getting data")
        df = pd.read_csv(url)
        df.to_csv(cache_fn, index=False)
    print("Using cache")
    df = spark.read.csv(str(cache_fn), header=True)
    return df

In [ ]:
url = 'http://biostat.mc.vanderbilt.edu/wiki/pub/Main/DataSets/titanic3.csv'

In [ ]:
df_raw = extract_titanic_data(url)

In [ ]:
type(df_raw)

In [ ]:
df_raw.show(10)

In [ ]:
df_raw.printSchema()

In [ ]:
# Rename home.dest column
df_raw = df_raw.withColumnRenamed('home.dest','home_dest')
df_raw.printSchema()

In [ ]:
df_raw.count()

In [ ]:
# Train/test split
train_df, test_df = df_raw.randomSplit([0.7,0.3],seed=1234)

In [ ]:
#df_raw.randomSplit?

In [ ]:
train_df.count()

In [ ]:
from pyspark.sql.functions import rand, when
df_raw = df_raw.withColumn('train', when(rand(seed=1234) >= 0.3, True).otherwise(False))

In [ ]:
#rand?

In [ ]:
df_raw.select('train').groupby('train').count().show()

In [ ]:
df_raw.select('train').printSchema()

In [ ]:
import pyspark.sql.functions as f

In [ ]:
df_train = df_raw.filter(f.col('train') == True)

In [ ]:
df_train.count()

In [ ]:
df_test = df_raw.filter(f.col('train') != True)

In [ ]:
df_test.count()

## Ejercicio 2: Primer preproceso / EDA

In [ ]:
# Casteo de datos
df_raw.printSchema()

In [ ]:
def cast_cols(df, cols, new_type):
  for col in cols: 
     df = df.withColumn(col, df[col].cast(new_type()))
  return df

In [ ]:
integer_cols = ['survived', 'sibsp', 'parch', 'body']
float_cols = ['age', 'fare']

In [ ]:
df_raw = cast_cols(df_raw, integer_cols, IntegerType)
df_raw = cast_cols(df_raw, float_cols, DoubleType)

In [ ]:
df_raw.printSchema()

In [ ]:
survived_on_boat = df_raw.filter((f.col('boat').isNotNull()) & (f.col('survived') == 1)).count()
survived = df_raw.filter(f.col('survived') == 1).count()
survived_on_boat / survived

In [ ]:
import logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s-%(name)s-%(levelname)s: %(message)s',
    handlers=[logging.FileHandler('/dataset/titanic_spark.log'), logging.StreamHandler()],
)
logger = logging.getLogger(__name__)

In [ ]:
def _drop_unusable_cols(df, cols=[]):
    logger.info(
        f"Dropping the following {len(cols)} unusable columns:\n"
        f"{cols}"
    )
    df = df.drop(*cols)
    logger.info(
        f"Remaining {len(df.columns)} columns:\n {sorted(df.columns)}"
    )
    return df

In [ ]:
train_df = _drop_unusable_cols(train_df, cols=['boat', 'body', 'train'])

In [ ]:
train_df.columns, test_df.columns

In [ ]:
test_df = _drop_unusable_cols(test_df, cols=['boat', 'body', 'train'])

In [ ]:
train_df = train_df.withColumn('train_new', f.lit(True))
test_df = test_df.withColumn('train_new', f.lit(False))

In [ ]:
joined_df = train_df.unionByName(test_df)
joined_df.count()

In [ ]:
df = _drop_unusable_cols(df_raw, cols=['boat', 'body', 'ticket'])

In [ ]:
# Plotting
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df.select('survived').groupBy('survived').count().toPandas().set_index('survived').plot.bar()

## EDA y Ejercicio 3

In [ ]:
import seaborn as sns

In [ ]:
g = sns.heatmap(df.select('survived', 'age', 'parch', 'fare', 'sibsp').toPandas().corr(),
                annot=True, fmt = ".2f", cmap = "coolwarm")

In [ ]:
g = sns.FacetGrid(df.toPandas(), col='survived')
g = g.map(sns.distplot, 'age')

In [ ]:
survived_age = df.filter((f.col('survived') == 1) & 
                         (f.col('age').isNotNull())).select('age').toPandas()
not_survived_age = df.filter((f.col('survived') == 0) & (f.col('age').isNotNull())).select('age').toPandas()

In [ ]:
survived_age.squeeze()

In [ ]:
type(survived_age.squeeze())

In [ ]:
g = sns.kdeplot(not_survived_age.squeeze(), color='Red', shade=True)
g = sns.kdeplot(survived_age.squeeze(), color='Blue', shade=True)
g.set_xlabel('age')
g.set_ylabel('Frequency')
g.legend(['Not Survived', 'Survived'])

In [ ]:
fare_mean = df.select(f.mean(f.col('fare'))).first()[0]
fare_mean

In [ ]:
df.select('fare').fillna(fare_mean).filter(f.col('fare').isNull()).show()

In [ ]:
# Distribución de precio de boletos 
g = sns.distplot(df.select('fare').fillna(fare_mean).toPandas(), color='m')

In [ ]:
df.select('fare').filter(f.col('fare') == 0).show()

In [ ]:
import numpy as np

In [ ]:
@f.udf('double')
def np_log(x):
    if x is None:
        return 0
    return float(np.log(x + 1)) 

In [ ]:
df = df.withColumn('log_fare', np_log(df.fare))
df.select('fare', 'log_fare').show()

In [ ]:
df = df.drop('fare')

In [ ]:
df.select('age').groupBy('age').count().show()

In [ ]:
g = sns.distplot(df.select('log_fare').fillna(np.log(fare_mean)).toPandas(), color='m')

In [ ]:
g = sns.barplot(x='sex', y='survived', data=df.toPandas())
g = g.set_ylabel("Survival Probability")

In [ ]:
df.select('sex', 'survived').groupby('sex').mean().show()

In [ ]:
g = sns.catplot(x='pclass', y='survived', hue='sex', data=df.toPandas(),
                   height=6, kind='bar')
g = g.set_ylabels("survival probability")

## Ejercicio 4: Valores nulos y constantes

In [ ]:
type(df)

In [ ]:
df.select('age').groupBy('age').count().show()

In [ ]:
df.select(f.sum(f.col('age').isNull().cast('integer'))).show()

In [ ]:
df_nulls = df.select([f.sum(f.col(c).isNull().cast('integer') / df.count()).alias(c) for c in df.columns])

In [ ]:
df_nulls.show()

In [ ]:
null_cols = [c for c in df_nulls.columns if df_nulls.select(c).first()[0] >= 0.5]
null_cols

In [ ]:
def _drop_nulls(df, max_null_prop=0.5):
    logger.info(
        f"Dropping columns with null ratio greater than {max_null_prop * 100}%..."
    )
    df_nulls = df.select([f.sum(f.col(c).isNull().cast('integer') / df.count()).alias(c) 
                          for c in df.columns])
    logger.info(f"Null proportions:\n {df_nulls.show()}")
    null_cols = [c for c in df_nulls.columns if df_nulls.select(c).first()[0] > max_null_prop 
                 and c not in PROTECTED_COLS]
    logger.info(f"Dropping the following {len(null_cols)} columns:\n {null_cols}")
    df = df.drop(*null_cols)
    return df

In [ ]:
PROTECTED_COLS = ['survived', 'train']
df = _drop_nulls(df)

In [ ]:
df.toPandas().isnull().mean()

In [ ]:
df.toPandas().info()

In [ ]:
df.printSchema()

In [ ]:
df.toPandas().std()

In [ ]:
df.dtypes

In [ ]:
num_cols = [c for c,dtype in df.dtypes if dtype.startswith(('int', 'double'))]
num_cols

In [ ]:
df_std = df.select([f.stddev(f.col(c)).alias(c) for c in num_cols])
df_std.show()

In [ ]:
def _drop_std(df, min_std_dev=1.5e-2):
    num_cols = [c for c,dtype in df.dtypes if dtype.startswith(('int', 'double'))]
    df_std = df.select([f.stddev(f.col(c)).alias(c) for c in num_cols])
    low_variance_cols = [c for c in df_std.columns if df_std.select(c).first()[0] < min_std_dev 
                         and c not in PROTECTED_COLS]
    logger.info(
        f'Dropping the following {len(low_variance_cols)} columns '
        f'due to low variance:\n {low_variance_cols}'
    )
    df.drop(*low_variance_cols)
    return df

In [ ]:
df = _drop_std(df)

In [ ]:
def _get_typed_cols(df, col_type='cat'):
    assert col_type in ('cat', 'num')
    dtypes = ('int', 'double') if col_type == 'num' else ('string')
    typed_cols = [c for c,dtype in df.dtypes if dtype.startswith(dtypes) 
                  and c not in PROTECTED_COLS]
    return typed_cols

In [ ]:
# _get_typed_cols(df, col_type='foo')

In [ ]:
num_cols = _get_typed_cols(df, col_type='num')
cat_cols = _get_typed_cols(df, col_type='cat')
num_cols, cat_cols

In [ ]:
val_counts = df.filter(f.col('embarked').isNotNull()).select('embarked').groupBy('embarked').count().orderBy(f.desc('count'))
val_counts.show()

In [ ]:
val_counts.select('embarked').first()[0]

In [ ]:
df.toPandas()['age'].median()

In [ ]:
# df.approxQuantile?

In [ ]:
df.approxQuantile('age', [0.5], 0)[0]

In [ ]:
def _fill_nulls(df):
    for t in ['num', 'cat']:
        cols = _get_typed_cols(df, col_type=t)
        for c in cols:
            if t == 'num':
                median_val = df.approxQuantile(c, [0.5], 0)[0]
                df = df.fillna(median_val, subset=[c])
            else:
                val_counts = df.filter(f.col(c).isNotNull()).select(c).groupBy(c).count().orderBy(f.desc('count'))
                common_val = val_counts.select(c).first()[0]
                df = df.fillna(common_val, subset=[c])
    return df

In [ ]:
df.toPandas().info()

In [ ]:
df.select('embarked').groupBy('embarked').count().orderBy(f.desc('count')).show()

In [ ]:
df = _fill_nulls(df)

In [ ]:
df.toPandas().info()

In [ ]:
df.select('embarked').groupBy('embarked').count().orderBy(f.desc('count')).show()

In [ ]:
df.select('age').groupBy('age').count().orderBy(f.desc('count')).show()

## Ejercicio 5: Ingenieria de Atributos

In [ ]:
df.select('name').show(10)

In [ ]:
df.select(f.trim(f.split(f.split(f.col('name'), ', ')[1], '. ')[0]).alias('title')).show(5)

In [ ]:
df = df.withColumn('title', f.trim(f.split(f.split(df['name'], ', ')[1], '. ')[0]))

In [ ]:
df.select('title').show()

In [ ]:
title_valcounts = df.select('title').groupBy('title').count().orderBy(f.desc('count'))
title_valcounts.show(5)

In [ ]:
other_titles = [str(i.title) for i in title_valcounts.select('title').collect()][4:]
other_titles

In [ ]:
df = df.withColumn('title', when(df['title'].isin(*other_titles), 'other').otherwise(df['title']))

In [ ]:
df.select('title').groupBy('title').count().show()

In [ ]:
df = df.withColumn('title', when(df['title'] == 'Miss', 'Mrs').otherwise(df['title']))

In [ ]:
df.select('title').groupBy('title').count().show()

In [ ]:
df = df.drop('name')

In [ ]:
g = sns.countplot(df.select('title').toPandas().squeeze())

In [ ]:
g = sns.catplot(x='title',y='survived',data=df.toPandas(),kind="bar")

In [ ]:
df.printSchema()

In [ ]:
# Tamaño de familia
df = df.withColumn('family_size', df['parch'] + df['sibsp'] + 1)
df.select('parch', 'sibsp', 'family_size').show(10)

In [ ]:
df = df.withColumn('family_single', when(df['family_size'] == 1, 1).otherwise(0))
df = df.withColumn('family_small', when((df['family_size'] > 1) & (df['family_size'] <= 4), 1).otherwise(0))
df = df.withColumn('family_large', when(df['family_size'] > 4, 1).otherwise(0))
df.select('family_size', 'family_single', 'family_small', 'family_large').show(10)
df = df.drop('family_size')

In [ ]:
df.columns

In [ ]:
for fsize in ['single', 'small', 'large']:
    g = sns.catplot(x=f'family_{fsize}',y='survived',data=df.toPandas(),kind="bar")
    g = g.set_ylabels("Survival Probability")

## Ejercicio 6: Fitteo de regresión logistica

In [ ]:
# Data sintetica
df1 = spark.createDataFrame([
    (1, 10.1, 'a', 'i'),
    (0, 14.3, 'b', 'x'),
    (0, 3.0, 'c', 'iv'),
    (1, 2.5, 'c', 'iv'),
    (1, 5.4, 'b', 'i'),
    (0, 9.7, 'a', 'x')
], ['target', 'numerical', 'cat1', 'cat2'])
df1.show()

# Convertir strings en numericos
from pyspark.ml.feature import StringIndexer

cat_cols = ['cat1', 'cat2']
for cat in cat_cols:
    cat_suff = f'{cat}_num'
    if cat_suff not in df1.columns:
        indexer = StringIndexer(inputCol=cat, outputCol=cat_suff).fit(df1)
        df1 = indexer.transform(df1)
df1.show()

# Hacer OneHotEnconding
from pyspark.ml.feature import OneHotEncoderEstimator

encoder = OneHotEncoderEstimator(inputCols=['cat1_num', 'cat2_num'], outputCols=['cat1_vec', 'cat2_vec'])
ohem = encoder.fit(df1)
df1 = ohem.transform(df1)
df1.show()

# Armar columna de features
df1 = df1.select('target','numerical', 'cat1_vec', 'cat2_vec')
df1.show()
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=[c for c in df1.columns if c != 'target'], outputCol='features')
assembler.transform(df1).show()

In [ ]:
num_cols = _get_typed_cols(df, col_type='num')
cat_cols = _get_typed_cols(df, col_type='cat')
df.columns, cat_cols, num_cols

In [ ]:
from pyspark.ml.feature import StringIndexer

def _encode_categorical(df):
    cat_cols = _get_typed_cols(df, col_type='cat')
    logger.info(f"Categorical columns:\n {cat_cols}")
    encoded_cols = []
    for cat in cat_cols:
        cat_suff = f'{cat}_num'
        encoded_cols.append(cat_suff)
        if cat_suff not in df.columns:
            indexer = StringIndexer(inputCol=cat, outputCol=cat_suff).fit(df)
            df = indexer.transform(df)
    return df, encoded_cols

In [ ]:
df, encoded_cols = _encode_categorical(df)
df.show()

In [ ]:
feature_cols = num_cols + encoded_cols
feature_cols

In [ ]:
from pyspark.ml.feature import OneHotEncoderEstimator

ohe_cols = [f'{c}_vec' for c in encoded_cols]
encoder = OneHotEncoderEstimator(inputCols=encoded_cols, outputCols=ohe_cols)
ohem = encoder.fit(df)
df = ohem.transform(df)
df.show()
feature_cols = num_cols + ohe_cols

In [ ]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=feature_cols, outputCol='features')
df = assembler.transform(df)
df.select('features').show()

In [ ]:
df.columns

In [ ]:
df.select('train').take(2)

In [ ]:
train_data = df.filter(f.col('train') == True).select('survived', 'features')
test_data = df.filter(f.col('train') == False).select('survived', 'features')
train_data.show()

In [ ]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(labelCol='survived', featuresCol='features')
lrm = lr.fit(train_data)

In [ ]:
dir(lrm.summary)

In [ ]:
# Metricas de evaluacion (insample)
lrm.summary.accuracy, lrm.summary.areaUnderROC

In [ ]:
pred_df = lrm.transform(test_data)
pred_df.show()

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator(labelCol='survived')
evaluator.evaluate(pred_df)

## Ejercicio 7: árboles

In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier

dt = DecisionTreeClassifier(labelCol='survived', featuresCol='features')
dtm = dt.fit(train_data)

In [ ]:
pred_df = dtm.transform(test_data)
pred_df.show()

In [ ]:
evaluator.evaluate(pred_df)

In [ ]:
from pyspark.ml.classification import RandomForestClassifier

rf = RandomForestClassifier(labelCol='survived', featuresCol='features')
rfm = rf.fit(train_data)
pred_df = rfm.transform(test_data)
pred_df.show()

In [ ]:
evaluator.evaluate(pred_df)

In [ ]:
dir(rfm)

In [ ]:
rfm.featureImportances

In [ ]:
from itertools import chain
attrs = sorted(
    (attr['idx'], attr['name'])
    for attr in (
        chain(*pred_df.schema['features'].metadata['ml_attr']['attrs'].values())
    )
)
feat_import = [(name, rfm.featureImportances[idx]) for idx, name in attrs if rfm.featureImportances[idx]]
feat_import = pd.DataFrame(feat_import, columns=['feature', 'importance']).sort_values(by='importance', ascending=False)
feat_import.head(15)

In [ ]:
ax = feat_import[:20].plot(kind='bar')
ax.set_xticklabels(feat_import[:20]['feature'].tolist())

In [ ]:
spark.stop()